In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
import os

from os import listdir


import uproot3


rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

#import mplhep as hep
#hep.set_style(hep.style.ROOT)

/home/miguel/.local/lib/python3.8/site-packages/uproot3/__init__.py:127: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  warnings.warn(
/home/miguel/.local/lib/python3.8/site-packages/awkward0/__init__.py:12: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  warnings.warn(


In [2]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

plot_style_1A = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': False
}

plot_style_2A = {'alpha': 0.5, 'density': False}


In [3]:
def get_Dataframe(path, name='Data', tag='nom'):
    Files = listdir(path) 
    #print (Files)
    df = None
    for i, f in enumerate(Files):
        #if i>20: continue
        if name not in f: continue
        if tag not in f: continue
        filename = path+f
        print ('filename is' , filename)
        
        temp_file = uproot3.open(filename)
        
        hasTree = False 
        for key in temp_file[name].keys():
            if('minitree' in str(key)):
                hasTree=True
        if (not hasTree):
            print('file has not minitree, skipping')
            continue

        temp_tree = temp_file[name+'/minitree']

        
        temp_df = None
        
        if 'Data' not in name:
            try:
                temp_df   =  temp_tree.pandas.df(["jet*", "genjet*","Q2","gen_Q2","y",'gen_y',"e_*","gene*","tau*","gen_tau*"], entrystop=3e8,flatten=True)
                df = pd.concat([df,temp_df])
            except ValueError:
                print ('oops, there is a problem in flattening the TTree ')
        else:
            try:
                temp_df   =  temp_tree.pandas.df(["jet*","Q2","y","e_*","tau*"], entrystop=3e8,flatten=True) 
                df = pd.concat([df,temp_df])
            except ValueError:
                print ('oops, there is a problem in flattening the TTree ')
        
        #try:
        #    df.shape[0]
        #except ValueError:
        #    print('no valid dataframe')
    print('####################################################################')
    print('Dataframe has a total of ', df.shape[0], ' entries')
    print('####################################################################')

    return df

In [4]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

In [15]:
def applyCutsJets(df,isMC=False):
    temp = df
    #temp = applyCut(temp, 'abs(vertex_z)<25 and vertex_z!=0','abs(vertex_z)<25 and and vertex_z!=0')
    #temp = applyCut(temp, 'tau1b>0 and tau1b<1', '0<tau1b<1')
    
    temp['pass_reco'] = np.where(temp['jet_pt']>0, 1, 0)
    temp['pass_truth'] = np.where(temp['genjet_pt']>0, 1, 0)
    
    temp.eval('jet_px = jet_pt*cos(jet_phi)', inplace=True)
    temp.eval('jet_py = jet_pt*sin(jet_phi)', inplace=True)
    temp.eval('jet_pz = jet_pt*sinh(jet_eta)', inplace=True)

    temp.eval('jet_qt = sqrt( (jet_px + e_px)**2 + (jet_py + e_py)**2) ', inplace=True)
    temp.eval('jet_qtnorm = jet_qt/sqrt(Q2)', inplace=True)
    temp.eval('e_pt = sqrt(e_px*e_px + e_py*e_py)',inplace=True)
    temp.eval('e_phi = arctan(e_py/e_px)', inplace=True)
    temp.eval('e_theta = abs(arctan(e_py/e_pz))', inplace=True)
    temp.eval('jet_theta = abs(arctan(jet_py/jet_pz))', inplace=True)

    temp.eval('e_p = sqrt(e_px*e_px + e_py*e_py + e_pz*e_pz)', inplace=True)

    temp.eval('jet_phi = arctan(jet_py/jet_px)',inplace=True)
    temp.eval('jet_dphi = e_phi-jet_phi',inplace=True)
    temp.eval('logQ2= log(Q2)/2.3025850', inplace=True)
    temp.eval('Q = sqrt(Q2)', inplace=True)
    temp = applyCut(temp, '0.08 < y < 0.7', '0.08 < y < 0.7')
    temp = applyCut(temp, 'Q2>150', 'Q2>150')
    temp = applyCut(temp, 'pass_reco==0 | jet_pt>5.0', 'jet pT > 5 GeV')
    temp = applyCut(temp, 'pass_reco==0 | jet_eta>-1.0', 'jet eta > -1.0')
    temp = applyCut(temp, 'pass_reco==0 | jet_eta<2.5', 'jet eta < 2.5')

    if(isMC):
        temp = applyCut(temp, 'pass_truth>0', 'pass truth')
        temp.eval('gen_logQ2= log(gen_Q2)/2.3025850', inplace=True)   
        temp.eval('gen_Q    = sqrt(gen_Q2)', inplace=True)
        temp.eval('gene_pt = sqrt(gene_px*gene_px + gene_py*gene_py)',inplace=True)
        temp.eval('gene_p = sqrt(gene_px*gene_px + gene_py*gene_py + gene_pz*gene_pz)',inplace=True)
        temp.eval('gene_theta = abs(arctan(gene_py/gene_pz))', inplace=True)

        temp.eval('genjet_px = genjet_pt*cos(genjet_phi)', inplace=True)
        temp.eval('genjet_py = genjet_pt*sin(genjet_phi)', inplace=True)
        temp.eval('genjet_pz = genjet_pt*sinh(genjet_eta)', inplace=True)
        temp.eval('genjet_theta = abs(arctan(genjet_py/genjet_pz))', inplace=True)

        temp.eval('genjet_qt = sqrt( (genjet_px + gene_px)**2 + (genjet_py + gene_py)**2) ', inplace=True)
        temp.eval('genjet_qtnorm = genjet_qt/sqrt(gen_Q2)', inplace=True)
        temp.eval('gene_phi = arctan(gene_py/gene_px)', inplace=True)
        temp.eval('genjet_phi = arctan(genjet_py/genjet_px)',inplace=True)
        temp.eval('genjet_dphi = gene_phi-genjet_phi',inplace=True)
        
    #    temp.eval('genjet_qtnormept= genjet_qt/e_pt', inplace=True)
    #    temp.eval('genjet_qtnormjetpt= genjet_qt/genjet_pt', inplace=True)


    #df = applyCut(df, 'n_total>1', ' n>1')
    return temp

In [6]:
mc_name = 'Django'
altmc_name = 'Rapgap'

#altmc_name = 'Rapgap'
#mc_name = 'Django'

In [ ]:

path = '/home/miguel/data/hera/'
data = get_Dataframe(path, name='Data')

In [ ]:

sys_data = get_Dataframe(path, name='Data')

In [ ]:
data['pass_reco'] = np.where(data['jet_pt']>0, 1, 0)
sys_data['pass_reco'] = np.where(sys_data['jet_pt']>0, 1, 0)
print('Selecting data events\n')
data = applyCutsJets(data)


print('Selecting data events\n')
sys_data = applyCutsJets(sys_data)

In [ ]:
%%time
mc = get_Dataframe(path, name=mc_name)


In [ ]:
%%time


sys_mc = get_Dataframe('/home/miguel/21-06-21-05-18-June20_test2/out_ep0607/', name=mc_name, tag = 'sys_7.')

In [ ]:
mc['pass_reco'] = np.where(mc['jet_pt']>0, 1, 0)
mc['pass_truth'] = np.where(mc['genjet_pt']>0, 1, 0)

sys_mc['pass_reco'] = np.where(sys_mc['jet_pt']>0, 1, 0)
sys_mc['pass_truth'] = np.where(sys_mc['genjet_pt']>0, 1, 0)

#mc['pass_truth'] = np.where(mc['genjet_pt']*mc['Q2']>0, 1, 0)


In [ ]:
mc.keys()

In [ ]:
%%time

print('Selecting MC events\n')
mc   = applyCutsJets(mc, isMC=True)

print('Selecting MC events\n')
sys_mc   = applyCutsJets(sys_mc, isMC=True)

In [ ]:
for obs in ['y','gen_y','Q','gen_Q','jet_pt','genjet_pt','e_p','gene_p','e_theta','gene_theta','jet_theta','genjet_theta','jet_phi','genjet_phi','e_phi','gene_phi','jet_eta','genjet_eta','jet_dphi']:
    #if 'gen' in obs: continue
    print('MC: obs ' , obs , ' nominal = %2.2f'%mc[obs].median(), ' systematic = %2.2f'%sys_mc[obs].median(),' ========%2.1f '%(100*(1- (mc[obs].median()/ sys_mc[obs].median()) )),' %')
   

In [ ]:
mc.keys()
#plt.plot(mc['jet_pt'],bins=)
# label = {}
# label['test'] = 'repeat'
# label['sys0'] = 'HFS scale (in jet)'
# label['sys1'] = 'HFS scale (remainder)'
# label['sys6'] = 'lepton energy scale'
# label['sys9'] = 'lepton polar angle'
# label['model'] = 'Model'

In [ ]:
plt.hist(mc['jet_pt'],bins=100,range=(0,100),alpha=0.3,density=True,label='nominal MC')
plt.hist(sys_mc['jet_pt'],bins=100,range=(0,100),alpha=0.3,density=True, label='syst_7')
plt.yscale('log')
plt.legend()
plt.xlabel('jet pT')
plt.show()


In [ ]:
bins = np.logspace(np.log10(10),np.log10(100),7)
plt.hist(mc['genjet_pt'],bins=bins,alpha=0.3, density=True,label='nominal')
plt.hist(sys_mc['genjet_pt'],bins=bins,alpha=0.3,density= True,label='sys7')
plt.yscale('log')
plt.xlabel('gen jet pT ')
plt.legend()
plt.show()


In [ ]:
bins = np.logspace(np.log10(10),np.log10(100),7)

plt.hist(mc['e_pt'],bins=bins,alpha=0.3,density=True, label='nominal')
plt.hist(sys_mc['e_pt'],bins=bins,alpha=0.3,density=True, label='sys7')
plt.yscale('log')
plt.xlabel('reco e pT ')

plt.show()


In [ ]:
bins = np.logspace(np.log10(10),np.log10(100),7)

plt.hist(mc['gene_pt'],bins=bins,alpha=0.3,density=True, label='nominal')
plt.hist(sys_mc['gene_pt'],bins=bins,alpha=0.3,density=True, label='sys7')
plt.yscale('log')
plt.xlabel('gen e pT ')

plt.show()


In [ ]:
bins = np.logspace(np.log10(10),np.log10(100),7)
ynom, x  = np.histogram(mc['genjet_pt'],bins=bins)
ysys, x  = np.histogram(sys_mc['genjet_pt'],bins=bins)

xerr = (x[1:] - x[:-1])/2.0
print(ynom)
print(ysys)
print(ynom/sum(ynom))
print(ysys/sum(ysys))
print(np.divide(ynom/sum(ynom),ysys/sum(ysys)))

In [7]:
## Bens

In [10]:
mc_nominal = get_Dataframe('/home/miguel/data/hera/', name='Rapgap', tag='nominal')


filename is /home/miguel/data/hera/Rapgap_Eplus0607_116.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_8.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_101.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_107.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_131.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_103.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_117.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_4.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_136.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_112.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_144.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_150.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_115.nominal.root
filename is /home/miguel/data/hera/Rapgap_Eplus0607_126.nominal.root
filename is /home/miguel/data/hera/Rap

TypeError: applyCutsJets() got an unexpected keyword argument 'verbose'

In [16]:
mc_nominal   = applyCutsJets(mc_nominal, isMC=True)

0.08 < y < 0.7 24408405  fraction kept: 27.5
Q2>150 20851981  fraction kept: 85.4
jet pT > 5 GeV 20851981  fraction kept: 100.0
jet eta > -1.0 20225630  fraction kept: 97.0
jet eta < 2.5 20069668  fraction kept: 99.2
pass truth 20069668  fraction kept: 100.0


In [17]:
mc_sys7 = get_Dataframe('/home/miguel/21-06-21-05-18-June20_test2/out_ep0607/', name='Rapgap', tag='sys_7')
mc_sys7   = applyCutsJets(mc_sys7, isMC=True)

filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_117.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_143.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_104.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_11.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_133.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_114.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_140.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_107.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_135.sys_7.root
filename is /home/miguel/21-06-21-05-18-June20_test2/out_ep0607/Rapgap_Eplus0607_8.sys_7.root
filename is /home/miguel/21-06-21-05-18-Jun

In [21]:
binspt = np.logspace(np.log10(10),np.log10(100),7)
mc_nominal_truth = np.array(mc_nominal['pass_truth'])
mc_nominal_pT = mc_nominal[['genjet_pt']].to_numpy()
#mc_nominal_wgt = mc_nominal['wgt'].to_numpy()
nom,_= np.histogram(mc_nominal_pT[mc_nominal_truth==1][:,0],bins=binspt,density=True)
mc_sys7_truth = np.array(mc_sys7['pass_truth'])
mc_sys7_pT = mc_sys7[['genjet_pt']].to_numpy()
#mc_sys7_wgt = mc_sys7['wgt'].to_numpy()
sysval7,_= np.histogram(mc_sys7_pT[mc_sys7_truth==1][:,0],bins=binspt,density=True)
print(sysval7/nom)

[0.99690746 1.00196783 1.00293494 1.00403475 0.99772604 0.99530774]


In [22]:
mc_nominal.head()

jet_pt   jet_phi      jet_eta  jet_dphi        jet_z  \
entry subentry                                                              
2     0           16.420273  1.553756     0.587027 -3.044224     0.728761   
      1            9.641730 -1.456244     1.784029 -0.034224     0.161802   
4     0           14.567807 -0.377384    -0.395859 -0.524154     0.679185   
      1        -9999.000000  0.689416 -9999.000000 -1.590954 -9999.000000   
      2        -9999.000000  0.689416 -9999.000000 -1.590954 -9999.000000   

                genjet_pt  genjet_phi  genjet_eta  genjet_z          Q2  ...  \
entry subentry                                                           ...   
2     0         20.568708   -1.467425    0.821434  0.846955  788.595337  ...   
      1          5.473908   -1.346081    2.155561  0.054393  788.595337  ...   
4     0         14.085672   -0.397176   -0.393945  0.653461  212.734039  ...   
      1          5.879607    0.214298    0.837583  0.115509  212.734039  ...   
      2          3.192867   -1.526671    1.398180  0.033691  212.734039  ...   

                   gene_p  gene_theta  genjet_px  genjet_py  genjet_pz  \
entry subentry                                                           
2     0         27.624157    1.056548  -2.122435  20.458912  18.861061   
      1         27.624157    1.056548  -1.219744   5.336281  23.310341   
4     0         12.731216    0.669764 -12.989202   5.448562  -5.693624   
      1         12.731216    0.669764   5.745117   1.250364   5.521008   
      2         12.731216    0.669764  -0.140839   3.189759   6.067695   

                genjet_theta  genjet_qt  genjet_qtnorm  gene_phi  genjet_dphi  
entry subentry                                                                 
2     0             0.826013   3.541956       0.127100 -1.491610    -0.024185  
      1             0.225046  18.670387       0.669973 -1.491610    -0.145529  
4     0             0.763408   7.555617       0.523518 -0.901228    -0.504051  
      1             0.222717  12.774529       0.885129 -0.901228    -1.115525  
      2             0.483992   6.723284       0.465847 -0.901228     0.625444  

[5 rows x 50 columns]

In [24]:
mc_sys7.head()

jet_pt   jet_phi      jet_eta  jet_dphi        jet_z  \
entry subentry                                                              
1     0           11.777094  0.094490     0.343734  0.366656     0.498689   
      1        -9999.000000  0.689416 -9999.000000 -0.228270 -9999.000000   
2     0           10.150356 -0.628101    -0.874414  0.243133     0.823157   
      1        -9999.000000  0.689416 -9999.000000 -1.074385 -9999.000000   
      2        -9999.000000  0.689416 -9999.000000 -1.074385 -9999.000000   

                genjet_pt  genjet_phi  genjet_eta  genjet_z          Q2  ...  \
entry subentry                                                           ...   
1     0         15.394296    0.121294    0.399466  0.580330  423.160339  ...   
      1          3.550444   -1.381154    0.950017  0.103410  423.160339  ...   
2     0          9.054357   -0.614802   -0.888929  0.777271  175.563858  ...   
      1          3.813429    0.152606    1.466834  0.048580  175.563858  ...   
      2          3.147472   -0.111272    0.817555  0.088157  175.563858  ...   

                   gene_p  gene_theta  genjet_px  genjet_py  genjet_pz  \
entry subentry                                                           
1     0         22.037672    0.471405 -15.281191  -1.862666   6.314353   
      1         22.037672    0.471405   0.669286  -3.486791   3.903748   
2     0         14.433956    0.302535   7.396391  -5.222525  -9.151365   
      1         14.433956    0.302535   3.769111   0.579696   7.826741   
      2         14.433956    0.302535  -3.128007   0.349503   2.869622   

                genjet_theta  genjet_qt  genjet_qtnorm  gene_phi  genjet_dphi  
entry subentry                                                                 
1     0             0.286853   5.545360       0.271880  0.461665     0.340371  
      1             0.729040  16.009962       0.784942  0.461665     1.842819  
2     0             0.518584   2.099461       0.154850 -0.385274     0.229528  
      1             0.073931   6.259603       0.461690 -0.385274    -0.537880  
      2             0.121197  12.281495       0.905847 -0.385274    -0.274002  

[5 rows x 50 columns]